In [1]:
import math
import torch
import torch.nn as nn

Setting parameters

In [ ]:
class NativeCustomLSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.input_size=input_size
        self.hidden_size=hidden_size

        self.U_f=nn.Parameter(torch.Tensor(input_size, hidden_size))
        self.V_f=nn.Parameter(torch.Tensor(hidden_size, hidden_size))
        self.b_f=nn.Parameter(torch.Tensor(hidden_size))

        self.U_i=nn.Parameter(torch.Tensor(input_size, hidden_size))
        self.V_i=nn.Parameter(torch.Tensor(hidden_size, hidden_size))
        self.b_i=nn.Parameter(torch.Tensor(hidden_size))

        self.U_o=nn.Parameter(torch.Tensor(input_size, hidden_size))
        self.V_o=nn.Parameter(torch.Tensor(hidden_size, hidden_size))
        self.b_o=nn.Parameter(torch.Tensor(hidden_size))

        self.U_c=nn.Parameter(torch.Tensor(input_size, hidden_size))
        self.V_c=nn.Parameter(torch.Tensor(hidden_size, hidden_size))
        self.b_c=nn.Parameter(torch.Tensor(hidden_size))

        self.init_weights()
    
    def init_weights(self):
        stdv=1.0/math.sqrt(self.hidden_size)
        for weight in self.parameters():
            weight.data.uniform_(-stdv, stdv)
    
    def forward(self, x, initial_state=None):
        """Assume x.shape is in the form of (batch_size, sequence_size, input_size)"""
        batch_size, seq_size, _=x.size()
        hidden_seq=[]

        if initial_state is None:
            h_t = torch.zeros(batch_size, self.hidden_size).to(x.device)
            c_t = torch.zeros(batch_size, self.hidden_size).to(x.device)

        else:
            h_t, c_t = initial_state

        for t in range(seq_size):
            x_t = x[:, t, :] # input at time t
            i_t = torch.sigmoid(x_t @ self.U_i + h_t @ self.V_i + self.b_i)
            f_t = torch.sigmoid(x_t @ self.U_f + h_t @ self.V_f + self.b_f)
            g_t = torch.tanh(x_t @ self.U_c + h_t @ self.V_c +self.b_c) #tanh:can both increase and decrease the values
            c_t = f_t * c_t + i_t * g_t
            o_t = torch.sigmoid(x_t @ self.U_o + h_t @ self.V_o + self.b_o)
            h_t = o_t * torch.tanh(c_t)

            

